In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split

file_path = "dataR2.csv"
df = pd.read_csv(file_path)

X = df.drop(columns=["Classification"])  
y = df["Classification"]

# (80%) de treino e validaçao / (20%) de teste
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# 60% de treino / 20% de validação 
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, random_state=42, stratify=y_train_val)

print(f"Tamanho treino:      {len(X_train)}")
print(f"Tamanho validação:   {len(X_val)}")
print(f"Tamanho teste:       {len(X_test)}")

X_train.to_csv("X_train.csv", index=False)  



Tamanho treino:      69
Tamanho validação:   23
Tamanho teste:       24


In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import kruskal

#--------------------------feature selection --------------------------
#Kruskal-Wallis
kruskal_selected_features = []
p_values = {}
classes = np.unique(y_train)
for feature in X_train.columns:
    groups = []
    for have_c in classes:
        groups.append(X_train.loc[y_train == have_c, feature]) #faz uma lista dependendo do valor do classification

    stat, p = kruskal(*groups) #stat fica com o valor H e p fica com o p-value
    p_values[feature] = p
    
    # Selecionar features com diferença estatisticamente significativa  (5% significance level)
    if p < 0.05:
        kruskal_selected_features.append(feature)

print("kruskal_selected_features:", kruskal_selected_features)



# Reduzir datasets apenas às features selecionadas pelo Kruskal-Wallis
X_krus_train_sel = X_train[kruskal_selected_features]
X_krus_val_sel = X_val[kruskal_selected_features]
X_krus_test_sel = X_test[kruskal_selected_features]


#ROC-AUC
from sklearn.metrics import roc_auc_score
import numpy as np

roc_auc_scores = {}
roc_auc_selected_features_auc = []

for feature in X_train.columns:
    x_feat = X_train[feature].values

    # Calcular ROC-AUC
    auc = roc_auc_score(y_train, x_feat)    
    roc_auc_scores[feature] = auc
    
    # Critério de seleção (tipicamente AUC > 0.6 é considerado relevante nas aulas)
    if auc > 0.6:
        roc_auc_selected_features_auc.append(feature)

print("\nroc_auc_selected_features:", roc_auc_selected_features_auc)

# Reduzir datasets apenas às features selecionadas pelo ROC-AUC
X_auc_train_sel = X_train[roc_auc_selected_features_auc]
X_auc_val_sel   = X_val[roc_auc_selected_features_auc]
X_auc_test_sel  = X_test[roc_auc_selected_features_auc]





Ranked features (by H)
BMI --> 0.653 (p=0.419)
Age --> 0.127 (p=0.722)


KeyError: 'Glucose'